In [ ]:
using CSV, DataFrames
# demand matrix D (day of the week, season)
# n locations
# walking matrix W (for each location, foot traffic for day of the week/season)


# D : 1×28 demand vector (Float64)
# W : n×28 foot-traffic matrix (Float64)

In [ ]:
using JuMP
using HiGHS
using LinearAlgebra

function optimize_installations(D::Vector{Float64}, W::Matrix{Float64})

    n, T = size(W)  # n locations, T = 28 days
    @assert length(D) == T "Demand vector D must have length 28"

    model = Model(HiGHS.Optimizer)

    @variable(model, x[1:n], Bin) # int decision vars

    @objective(model, Min, sum(x[i] for i in 1:n)) # objective func

    for t in 1:T # constraint each day
        @constraint(model, sum(W[i, t] * x[i] for i in 1:n) ≥ 0.01 * D[t])
    end

    optimize!(model) # solve model

    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("Warning: Solver returned status $status")
    end

    # get sol
    chosen_locations = findall(i -> value(x[i]) > 0.5, 1:n)
    min_num_locations = length(chosen_locations)

    return min_num_locations, chosen_locations, model
end

# demand matrix D, foot traffic matrix W
min_locs, chosen, model = optimize_installations(D, W)